### Import Python packages 

In [3]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

### Creating list of filepaths to process original event csv data files

In [4]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

### Creating the the tables using the data from the `event_data_file_new.csv` file

### Creating a Cluster

In [7]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

try: 
    session = cluster.connect()
except Exception as e:
    print(e)

session = cluster.connect()

### Create Keyspace

In [8]:
query = "CREATE KEYSPACE IF NOT EXISTS sparkify \
        WITH REPLICATION = \
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"
try:
    session.execute(query)

except Exception as e:
    print(e)

### Set Keyspace

In [9]:
session.set_keyspace('sparkify')

## Queries:

### Query One: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

<br>

In [10]:
query = "CREATE TABLE IF NOT EXISTS session_library ( \
        session_id text, \
        item_in_session text, \
        artist text, \
        song_title text, \
        song_length float, \
        PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)

except Exception as e:
    print(e)                   

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_library (session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[8], line[3], line[0], line[9], float(line[5])))

#### Verify data was inserted to table

In [12]:
try:
    rows = session.execute("SELECT COUNT(*) FROM session_library")
    
    for row in rows:
        print(row)
except Exception as e:
    print(e)
    print("")

Row(count=6820)


### Query Two: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
<br>


In [14]:
query = "CREATE TABLE IF NOT EXISTS user_library ( user_id text, \
        session_id text, \
        item_in_session text, \
        artist text, \
        song_title text,\
        user_first_name text, \
        user_last_name text, \
        PRIMARY KEY (user_id, session_id, item_in_session) \
    ) WITH CLUSTERING ORDER BY (session_id ASC, item_in_session ASC)"

try:
    session.execute(query)

except Exception as e:
    print(e)                      

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_library (user_id, session_id, item_in_session, artist, song_title, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query, (line[10], line[8], line[3], line[0], line[9], line[1], line[4]))

#### Validate data was inserted to table

In [16]:
try:
    rows = session.execute("SELECT COUNT(*) FROM user_library")
    for row in rows:
        print(row)
except Exception as e:
    print(e)

Row(count=6820)


### Query Four: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

<br>

In [17]:
query = "CREATE TABLE IF NOT EXISTS song_library ( song_title text, \
        user_id text, \
        user_first_name text, \
        user_last_name text, \
        PRIMARY KEY (song_title, user_id) \
    ) "

try:
    session.execute(query)

except Exception as e:
    print(e)                   

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library (song_title, user_id, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], line[10], line[1], line[4]))

#### Validate data was inserted to table

In [19]:
try:
    rows = session.execute("SELECT COUNT(*) FROM song_library")
    
    for row in rows:
        print(row)
except Exception as e:
    print(e)
    print("")

Row(count=6618)


## Query Results

In [20]:
print("Query #1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4")
print("")

try:
    rows = session.execute("SELECT session_id, item_in_session, artist, song_title, song_length\
                           FROM session_library \
                           WHERE session_id='338' \
                           AND item_in_session='4'")
    for row in rows:
        print(row)
except Exception as e:
    print(e)

print("")

print("Query #2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182")
print("")

try:
    rows = session.execute("SELECT artist, song_title, user_id, session_id, item_in_session \
                            FROM user_library \
                            WHERE user_id='10' AND session_id='182'")
    for row in rows:
        print(row)
except Exception as e:
    print(e)

print("")

print("Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'")
print("")

try:
    rows = session.execute("SELECT song_title, user_id, user_first_name, user_last_name \
                            FROM song_library \
                            WHERE song_title='All Hands Against His Own'")
    for row in rows:
        print(row)
except Exception as e:
    print(e)

Query #1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

Row(session_id='338', item_in_session='4', artist='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)

Query #2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Row(artist='Down To The Bone', song_title="Keep On Keepin' On", user_id='10', session_id='182', item_in_session='0')
Row(artist='Three Drives', song_title='Greece 2000', user_id='10', session_id='182', item_in_session='1')
Row(artist='Sebastien Tellier', song_title='Kilometer', user_id='10', session_id='182', item_in_session='2')
Row(artist='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_id='10', session_id='182', item_in_session='3')

Query 3: Give me every user name (first and last) in my music app history who lis

### Drop the tables

In [21]:
try:
    session.execute("DROP TABLE IF EXISTS session_library")
except Exception as e:
    print(e)

try:
    session.execute("DROP TABLE IF EXISTS user_library")
except Exception as e:
    print(e)

try:
    session.execute("DROP TABLE IF EXISTS song_library")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()

## OPTIONAL: Question for the reviewer
 
If you have any question about the starter code or your own implementation, please add it in the cell below. 

For example, if you want to know why a piece of code is written the way it is, or its function, or alternative ways of implementing the same functionality, or if you want to get feedback on a specific part of your code or get feedback on things you tried but did not work.

Please keep your questions succinct and clear to help the reviewer answer them satisfactorily. 

> **_Your question_**